<a href="https://colab.research.google.com/github/giorginolab/protein_bert/blob/master/ProteinBERT_Toni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ProteinBERT demo

This uses the model and weights provided by

> Brandes N, Ofer D, Peleg Y, Rappoport N, Linial M. ProteinBERT: A universal deep-learning model of protein sequence and function. bioRxiv. 2021 May 25;2021.05.24.445464.  https://www.biorxiv.org/content/10.1101/2021.05.24.445464v1 

Code based on the repository, https://github.com/nadavbra/protein_bert . 

Adapted for Google Colab by Toni Giorgino, www.giorginolab.it .


# Setup

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
!nvidia-smi

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3749876317132647526
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7744061440
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 7242276252769984084
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:02:00.0, compute capability: 6.1"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 7744061440
locality {
  bus_id: 1
  links {
    link {
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 8218700750548764620
physical_device_desc: "device: 1, name: GeForce GTX 1080, pci bus id: 0000:03:00.0, compute capability: 6.1"
]
Wed Jul 14 15:19:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------

In [2]:
# The examples in this notebook use a set of nine benchmarks described in the publication.
# These benchmarks can be downloaded via FTP from: ftp.cs.huji.ac.il/users/nadavb/protein_bert/protein_benchmarks
# Download the benchmarks into a directory on your machine and set the following variable to the path of that directory.
import os
BENCHMARKS_DIR = os.path.join(os.getcwd(), "protein_benchmarks")


# Fine-tune the model for the signal peptide benchmark

# Run all benchmarks

Do not run - takes too much time.

In [ ]:
import os

import pandas as pd
from IPython.display import display

from tensorflow import keras

from sklearn.model_selection import train_test_split

from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len, log
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

BENCHMARKS = [
    # name, output_type
    ('signalP_binary', OutputType(False, 'binary')),
    ('fluorescence', OutputType(False, 'numeric')),
    ('remote_homology', OutputType(False, 'categorical')),
    ('stability', OutputType(False, 'numeric')),
    ('scop', OutputType(False, 'categorical')),
    ('secondary_structure', OutputType(True, 'categorical')),
    ('disorder_secondary_structure', OutputType(True, 'binary')),
    ('ProFET_NP_SP_Cleaved', OutputType(False, 'binary')),
    ('PhosphositePTM', OutputType(True, 'binary')),
]

settings = {
    'max_dataset_size': None,
    'max_epochs_per_stage': 40,
    'seq_len': 512,
    'batch_size': 32,
    'final_epoch_seq_len': 1024,
    'initial_lr_with_frozen_pretrained_layers': 1e-02,
    'initial_lr_with_all_layers': 1e-04,
    'final_epoch_lr': 1e-05,
    'dropout_rate': 0.5,
    'training_callbacks': [
        keras.callbacks.ReduceLROnPlateau(patience = 1, factor = 0.25, min_lr = 1e-05, verbose = 1),
        keras.callbacks.EarlyStopping(patience = 2, restore_best_weights = True),
    ],
}

####### Uncomment for debug mode
# settings['max_dataset_size'] = 500
# settings['max_epochs_per_stage'] = 1

def run_benchmark(benchmark_name, pretraining_model_generator, input_encoder, pretraining_model_manipulation_function = None):
    
    log('========== %s ==========' % benchmark_name)  
    
    output_type = get_benchmark_output_type(benchmark_name)
    log('Output type: %s' % output_type)
    
    train_set, valid_set, test_set = load_benchmark_dataset(benchmark_name)        
    log(f'{len(train_set)} training set records, {len(valid_set)} validation set records, {len(test_set)} test set records.')
    
    if settings['max_dataset_size'] is not None:
        log('Limiting the training, validation and test sets to %d records each.' % settings['max_dataset_size'])
        train_set = train_set.sample(min(settings['max_dataset_size'], len(train_set)), random_state = 0)
        valid_set = valid_set.sample(min(settings['max_dataset_size'], len(valid_set)), random_state = 0)
        test_set = test_set.sample(min(settings['max_dataset_size'], len(test_set)), random_state = 0)
    
    if output_type.is_seq or output_type.is_categorical:
        train_set['label'] = train_set['label'].astype(str)
        valid_set['label'] = valid_set['label'].astype(str)
        test_set['label'] = test_set['label'].astype(str)
    else:
        train_set['label'] = train_set['label'].astype(float)
        valid_set['label'] = valid_set['label'].astype(float)
        test_set['label'] = test_set['label'].astype(float)
        
    if output_type.is_categorical:
        
        if output_type.is_seq:
            unique_labels = sorted(set.union(*train_set['label'].apply(set)) | set.union(*valid_set['label'].apply(set)) | \
                    set.union(*test_set['label'].apply(set)))
        else:
            unique_labels = sorted(set(train_set['label'].unique()) | set(valid_set['label'].unique()) | set(test_set['label'].unique()))
            
        log('%d unique lebels.' % len(unique_labels))
    elif output_type.is_binary:
        unique_labels = [0, 1]
    else:
        unique_labels = None
        
    output_spec = OutputSpec(output_type, unique_labels)
    model_generator = FinetuningModelGenerator(pretraining_model_generator, output_spec, pretraining_model_manipulation_function = \
            pretraining_model_manipulation_function, dropout_rate = settings['dropout_rate'])
    finetune(model_generator, input_encoder, output_spec, train_set['seq'], train_set['label'], valid_set['seq'], valid_set['label'], \
            seq_len = settings['seq_len'], batch_size = settings['batch_size'], max_epochs_per_stage = settings['max_epochs_per_stage'], \
            lr = settings['initial_lr_with_all_layers'], begin_with_frozen_pretrained_layers = True, lr_with_frozen_pretrained_layers = \
            settings['initial_lr_with_frozen_pretrained_layers'], n_final_epochs = 1, final_seq_len = settings['final_epoch_seq_len'], \
            final_lr = settings['final_epoch_lr'], callbacks = settings['training_callbacks'])
    
    finetuned_model = model_generator.create_model(settings['seq_len'])
    finetuned_model.save(benchmark_name)
    finetuned_model.save_weights(benchmark_name)
    
    for dataset_name, dataset in [('Training-set', train_set), ('Validation-set', valid_set), ('Test-set', test_set)]:
        
        log('*** %s performance: ***' % dataset_name)
        results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, output_spec, dataset['seq'], dataset['label'], \
                start_seq_len = settings['seq_len'], start_batch_size = settings['batch_size'])
    
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(results)
        
        if confusion_matrix is not None:
            with pd.option_context('display.max_rows', 16, 'display.max_columns', 10):
                log('Confusion matrix:')
                display(confusion_matrix)
                
    return model_generator

def load_benchmark_dataset(benchmark_name):
    
    train_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.train.csv' % benchmark_name)
    valid_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.valid.csv' % benchmark_name)
    test_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.test.csv' % benchmark_name)
    
    train_set = pd.read_csv(train_set_file_path).dropna().drop_duplicates()
    test_set = pd.read_csv(test_set_file_path).dropna().drop_duplicates()
          
    if os.path.exists(valid_set_file_path):
        valid_set = pd.read_csv(valid_set_file_path).dropna().drop_duplicates()
    else:
        log(f'Validation set {valid_set_file_path} missing. Splitting training set instead.')
        train_set, valid_set = train_test_split(train_set, stratify = train_set['label'], test_size = 0.1, random_state = 0)
    
    return train_set, valid_set, test_set

def get_benchmark_output_type(benchmark_name):
    for name, output_type in BENCHMARKS:
        if name == benchmark_name:
            return output_type
        
pretrained_model_generator, input_encoder = load_pretrained_model()

for benchmark_name, _ in BENCHMARKS:
    run_benchmark(benchmark_name, pretrained_model_generator, input_encoder, 
                  pretraining_model_manipulation_function = get_model_with_hidden_layers_as_outputs)
    print("Skipping - uncomment if you are sure")
        
log('Done.')

[2021_07_14-15:20:00] ========== signalP_binary ==========
[2021_07_14-15:20:00] Output type: global binary
[2021_07_14-15:20:00] Validation set /home/toni/work/protein_bert/protein_benchmarks/signalP_binary.valid.csv missing. Splitting training set instead.
[2021_07_14-15:20:00] 14945 training set records, 1661 validation set records, 4152 test set records.
[2021_07_14-15:20:00] Training set: Filtered out 0 of 14945 (0.0%) records of lengths exceeding 510.
[2021_07_14-15:20:01] Validation set: Filtered out 0 of 1661 (0.0%) records of lengths exceeding 510.
[2021_07_14-15:20:01] Training with frozen pretrained layers...


/home/toni/Apps/anaconda3/envs/proteinbert/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/40
468/468 [==============================] - 42s 72ms/step - loss: 0.0946 - val_loss: 0.0653
Epoch 2/40
468/468 [==============================] - 32s 68ms/step - loss: 0.0744 - val_loss: 0.0683

Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.0024999999441206455.
Epoch 3/40
468/468 [==============================] - 32s 68ms/step - loss: 0.0593 - val_loss: 0.0649
Epoch 4/40
468/468 [==============================] - 32s 68ms/step - loss: 0.0563 - val_loss: 0.0629
Epoch 5/40
468/468 [==============================] - 32s 68ms/step - loss: 0.0597 - val_loss: 0.0609
Epoch 6/40
468/468 [==============================] - 32s 68ms/step - loss: 0.0555 - val_loss: 0.0619

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0006249999860301614.
Epoch 7/40
468/468 [==============================] - 32s 68ms/step - loss: 0.0507 - val_loss: 0.0616

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.00015624999650754035.
[2021_07_14-15:23:59] Training the entire 

/home/toni/Apps/anaconda3/envs/proteinbert/lib/python3.9/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: signalP_binary/assets


...

[2021_07_14-15:34:03] Confusion matrix:


,0,1
0,3427,51
1,24,650


Skipping - uncomment if you are sure
[2021_07_14-15:34:03] ========== fluorescence ==========
[2021_07_14-15:34:03] Output type: global numeric
[2021_07_14-15:34:03] 21446 training set records, 5362 validation set records, 27217 test set records.
[2021_07_14-15:34:03] Training set: Filtered out 0 of 21446 (0.0%) records of lengths exceeding 510.
[2021_07_14-15:34:05] Validation set: Filtered out 0 of 5362 (0.0%) records of lengths exceeding 510.
[2021_07_14-15:34:05] Training with frozen pretrained layers...


/home/toni/Apps/anaconda3/envs/proteinbert/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/40
671/671 [==============================] - 62s 81ms/step - loss: 0.7058 - val_loss: 0.5614
Epoch 2/40
671/671 [==============================] - 52s 77ms/step - loss: 0.6405 - val_loss: 0.5859

Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.0024999999441206455.
Epoch 3/40
671/671 [==============================] - 52s 77ms/step - loss: 0.5420 - val_loss: 0.4023
Epoch 4/40
671/671 [==============================] - 52s 77ms/step - loss: 0.5108 - val_loss: 0.4001
Epoch 5/40
671/671 [==============================] - 52s 77ms/step - loss: 0.5025 - val_loss: 0.3978
Epoch 6/40
671/671 [==============================] - 52s 77ms/step - loss: 0.4974 - val_loss: 0.3892
Epoch 7/40
671/671 [==============================] - 52s 77ms/step - loss: 0.4953 - val_loss: 0.4319

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.0006249999860301614.
Epoch 8/40
671/671 [==============================] - 52s 77ms/step - loss: 0.4779 - val_loss: 0.3845
Epoch 9/40
671/671 [==

# Predict an arbitrary sequence

In [31]:
my_sequence = "AAAAA"
my_seq_len = 512
my_sequence_encoded = input_encoder.encode_X([my_sequence],my_seq_len)

finetuned_model = model_generator.create_model(my_seq_len)
finetuned_model.predict(my_sequence_encoded)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


array([[0.00042532]], dtype=float32)

In [32]:
my_sequence_encoded

[array([[23,  0,  0,  0,  0,  0, 24, 25, 25, 25, 25, 25, 25, 25, 25, 25,
         25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,
         25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,
         25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,
         25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,
         25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,
         25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,
         25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,
         25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,
         25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,
         25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,
         25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,
         25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,
         25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25

# Visualizing the attention layers

You can run this only after you have fine-tuned the model on a benchmark (e.g. signal peptide) and obtained *model_generator*.

In [18]:
finetuned_model.save("finetuned")
finetuned_model.save_weights("finetuned_weights")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: finetuned/assets


INFO:tensorflow:Assets written to: finetuned/assets


In [19]:
%ls -l 

total 62568
drwxr-xr-x 2 root root     4096 Jul 14 09:28  bin/
drwxr-xr-x 5 root root     4096 Jul 14 09:28  build/
-rw-r--r-- 1 root root       91 Jul 14 10:12  checkpoint
drwxr-xr-x 2 root root     4096 Jul 14 09:28  dist/
drwxr-xr-x 4 root root     4096 Jul 14 10:12  finetuned/
-rw-r--r-- 1 root root 63994839 Jul 14 10:12  finetuned_weights.data-00000-of-00001
-rw-r--r-- 1 root root     9163 Jul 14 10:12  finetuned_weights.index
drwxr-xr-x 4 root root     4096 Jul 14 09:30  proteinbert/
-rw-r--r-- 1 root root    18446 Jul 14 09:28 'ProteinBERT demo.ipynb'
drwxr-xr-x 2 root root     4096 Jul 14 09:28  protein_bert.egg-info/
-rw-r--r-- 1 root root     7624 Jul 14 09:28  README.rst
-rw-r--r-- 1 root root      711 Jul 14 09:28  setup.py
